In [24]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo

In [25]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [26]:
# Define database and collection
db = client.nhl_db
collection = db.articles

In [27]:
# URL of page to be scraped
url = 'https://www.nhl.com/news'

In [28]:
# Retrieve page with the requests module
response = requests.get(url)

In [29]:
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')

In [34]:
# Retrieve the parent divs for all articles
results = soup.find_all("div", class_= "article-item__top")
# loop over results to get article data
for result in results:
    
    try:
        # scrape the article header 
        header = result.find('h1', class_= "article-item__headline").text
    
        # scrape the article subheader
        subheader = result.find('h2', class_= "article-item__subheader").text
    
        # scrape the datetime
        date = result.find("span", class_ = "article-item__date")["data-date"]
        # get only the date from the datetime
        clean_date = date.split("T")[0]
        
        if (header and subheader and date):
            # print article data
            print('-----------------')
            print(header)
            print(subheader)
            print(date)

            # Dictionary to be inserted into MongoDB
            post = {
                'header': header,
                'subheader': subheader,
                'date': date
            }
    
            collection.insert_one(post)

    except Exception as e:
        print(e)
    
    # Insert dictionary into MongoDB as a document
listings = db.articles.find()

for listing in listings:
    print(listing["header"])
    print(listing["subheader"])

-----------------
Vasilevskiy sets Lightning point record in win against Penguins
Makes 29 saves for longest streak by goalie with 17; Kucherov has goal, assist for Tampa Bay
2020-02-06T22:00:40-0500
-----------------
Petry scores in overtime, Canadiens defeat Ducks
Defenseman's goal at 25 seconds produces sixth win in eight games
2020-02-06T21:46:46-0500
-----------------
Avalanche top Senators for fifth win in six games
Makar has goal, assist for Colorado; Ottawa has lost four straight
2020-02-06T22:14:37-0500
-----------------
Bellows scores first two NHL goals to lift Islanders past Kings
Second one breaks tie with 7:50 left in third; New York extends point streak to five
2020-02-06T21:46:12-0500
-----------------
Fantasy goalie top 25 rankings for 2019-20
Bobrovsky among 10 best for Panthers; Merzlikins continues to trend up
2020-02-06T09:00:00-0500
-----------------
Super 16: Lightning move past Penguins, Blues to take third in rankings
Blue Jackets climb four spots; Panthers fal

In [33]:
# Display the MongoDB records created above
articles = db.articles.find()
for article in articles:
    print(article)